# Derivation

In [1]:
using Pkg
Pkg.activate(".")
using SymPy

  Activating project at `c:\Users\tim\Documents\2025examples\02-27`


In [2]:
t, t1, t2, t3, t4 = symbols("t, t_1, t_2, t_3, t_4", real=true)
x, x1, x2, x3, x4 = symbols("x, x_1, x_2, x_3, x_4", real=true)
h = symbols("h", real=true, positive=true)
function l(t, T, j)
    n = length(T)
    out = 1
    for m = 1:n
        if m != j
            out *= (t - T[m]) / (T[j] - T[m])
        end
    end
    return out
end
Lag(T,X,t) = sum(X[j] * l(t, T, j) for j in 1:length(X));

## Trap rule

In [3]:
T = [t1,t2]
X = [x1,x2]
I_trap = integrate( Lag(T,X,t), (t,t1,t2)) |> simplify

  t_1*x_1   t_1*x_2   t_2*x_1   t_2*x_2
- ------- - ------- + ------- + -------
     2         2         2         2   

In [4]:
I_trap(t2=>t1+h) |> simplify

h*(x_1 + x_2)
-------------
      2      

## Simpson's Rule

In [5]:
T = [t1, t2, t3]
X = [x1, x2, x3]
I_3 = integrate(Lag(T, X, t), (t, t1, t3)) |> simplify

     3          3            2                2                2              
- t_1 *x_2 + t_1 *x_3 - 2*t_1 *t_2*x_1 - 4*t_1 *t_2*x_3 + 2*t_1 *t_3*x_1 + 3*t
------------------------------------------------------------------------------
                                                                              
                                                                              

  2              2                    2                2                      
_1 *t_3*x_2 + t_1 *t_3*x_3 + 3*t_1*t_2 *x_1 + 3*t_1*t_2 *x_3 - 2*t_1*t_2*t_3*x
------------------------------------------------------------------------------
                                                                /             
                                                              6*\t_1*t_2 - t_1

                                2                2                2           
_1 + 2*t_1*t_2*t_3*x_3 - t_1*t_3 *x_1 - 3*t_1*t_3 *x_2 - 2*t_1*t_3 *x_3 - 3*t_
--------------------------------------------------

In [6]:
I_3(t2=>t1+h, t3=>t1+2h) |> simplify

h*(x_1 + 4*x_2 + x_3)
---------------------
          3          

## Gauss-Legendre Quadrature

In [7]:
w1, w2, w3, w4 = symbols("w1, w2, w3, w4", real=true)
a0, a1, a2, a3, a4, a5 = symbols("a0, a1, a2, a3, a4, a5", real=true);

### n = 2

In [8]:
f = a0 + a1*t + a2*t^2 + a3*t^3
Intf = integrate(f,(t, -1, 1))

       2*a2
2*a0 + ----
        3  

In [9]:
GL2 = w1*f(t=>t1) + w2*f(t=>t2)

   /                    2         3\      /                    2         3\
w1*\a0 + a1*t_1 + a2*t_1  + a3*t_1 / + w2*\a0 + a1*t_2 + a2*t_2  + a3*t_2 /

In [10]:
eq = Intf - GL2

       2*a2      /                    2         3\      /                    2
2*a0 + ---- - w1*\a0 + a1*t_1 + a2*t_1  + a3*t_1 / - w2*\a0 + a1*t_2 + a2*t_2 
        3                                                                     

         3\
 + a3*t_2 /
           

In [11]:
eqset = [diff(eq,a) for a in [a0,a1,a2,a3]]

4-element Vector{Sym{PyCall.PyObject}}:
               -w1 - w2 + 2
           -t_1*w1 - t_2*w2
 -t_1^2*w1 - t_2^2*w2 + 2/3
       -t_1^3*w1 - t_2^3*w2

In [12]:
sol = solve(eqset, [t1,t2,w1,w2], dict=true)[1]

Dict{Sym{PyCall.PyObject}, Sym{PyCall.PyObject}} with 4 entries:
  t_2 => sqrt(3)/3
  t_1 => -sqrt(3)/3
  w2  => 1
  w1  => 1

### n = 3

In [13]:
f = a0 + a1 * t + a2 * t^2 + a3 * t^3 + a4*t^4 + a5*t^5
Intf = integrate(f, (t, -1, 1))

       2*a2   2*a4
2*a0 + ---- + ----
        3      5  

In [14]:
GL3 = w1 * f(t => t1) + w2 * f(t => t2) + w3 * f(t => t3)

   /                    2         3         4         5\      /               
w1*\a0 + a1*t_1 + a2*t_1  + a3*t_1  + a4*t_1  + a5*t_1 / + w2*\a0 + a1*t_2 + a

     2         3         4         5\      /                    2         3   
2*t_2  + a3*t_2  + a4*t_2  + a5*t_2 / + w3*\a0 + a1*t_3 + a2*t_3  + a3*t_3  + 

      4         5\
a4*t_3  + a5*t_3 /

In [15]:
eq = Intf - GL3

       2*a2   2*a4      /                    2         3         4         5\ 
2*a0 + ---- + ---- - w1*\a0 + a1*t_1 + a2*t_1  + a3*t_1  + a4*t_1  + a5*t_1 / 
        3      5                                                              

     /                    2         3         4         5\      /             
- w2*\a0 + a1*t_2 + a2*t_2  + a3*t_2  + a4*t_2  + a5*t_2 / - w3*\a0 + a1*t_3 +
                                                                              

       2         3         4         5\
 a2*t_3  + a3*t_3  + a4*t_3  + a5*t_3 /
                                       

In [16]:
eqset = [diff(eq, a) for a in [a0, a1, a2, a3, a4, a5]]

6-element Vector{Sym{PyCall.PyObject}}:
                     -w1 - w2 - w3 + 2
             -t_1*w1 - t_2*w2 - t_3*w3
 -t_1^2*w1 - t_2^2*w2 - t_3^2*w3 + 2/3
       -t_1^3*w1 - t_2^3*w2 - t_3^3*w3
 -t_1^4*w1 - t_2^4*w2 - t_3^4*w3 + 2/5
       -t_1^5*w1 - t_2^5*w2 - t_3^5*w3

In [17]:
sol = solve(eqset, [t1, t2, t3, w1, w2, w3], dict=true)[1]

Dict{Sym{PyCall.PyObject}, Sym{PyCall.PyObject}} with 6 entries:
  t_2 => -sqrt(15)/5
  w3  => 5/9
  t_3 => sqrt(15)/5
  t_1 => 0
  w2  => 5/9
  w1  => 8/9